In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Create X, y and split dataset

y = dataset['expenses'].tolist()
dataset.drop(['expenses'], axis=1, inplace=True)

# Category
dataset['sex'] = dataset['sex'].astype('category')
dataset['sex'] = dataset['sex'].cat.codes

dataset['smoker'] = dataset['smoker'].astype('category')
dataset['smoker'] = dataset['smoker'].cat.codes

dataset['region'] = dataset['region'].astype('category')
dataset['region'] = dataset['region'].cat.codes

X = np.array(dataset)
y = np.array(y).reshape(-1,1)

train_dataset, test_dataset, train_labels, test_labels = train_test_split(
    X, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(train_dataset)
train_dataset, test_dataset = scaler.transform(train_dataset), scaler.transform(test_dataset)

In [ ]:
# Create model
import tensorflow as tf
from keras.callbacks import Callback

# Define Classfor print result for each 100 epochs
log_p = 'log.txt'
class print_training_on_text_every_100_epochs_Callback(Callback):
    def __init__(self, logpath):
        self.logpath = logpath

    def on_epoch_end(self, epoch, logs=None):
        with open(self.logpath, "a") as writefile:  # put log_path here
            #with open(writefile):
              if (int(epoch) % 100) == 0:
                  print(
                      f"Epoch: {epoch:>3}"
                      + f" | Loss: {logs['loss']:.4e}"
                      + f" | MAE: {logs['mae']:.4e}"
                      + f" | Validation loss: {logs['val_loss']:.4e}"
                      + f" | Validation MAE: {logs['val_mae']:.4e}"
                  )
                  writefile.write("\n")

inputs = keras.Input(shape=(train_dataset.shape[1],), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
x = layers.Dense(32, activation="relu", name="dense_23")(x)
outputs = layers.Dense(1, name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-3,
    decay_steps=10000,
    decay_rate=0.9)

my_callbacks = [
    print_training_on_text_every_100_epochs_Callback(logpath=log_p),
    tf.keras.callbacks.EarlyStopping(patience=2000),
    tf.keras.callbacks.ModelCheckpoint(filepath='best-weight.keras', save_best_only=True),
    tf.keras.callbacks.TensorBoard(log_dir='logs'),
]

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='mse', metrics=['mae', 'mse'])

model.fit(train_dataset, train_labels,
          batch_size=train_dataset.shape[0],
          epochs=10000, verbose=0,
          validation_data=(test_dataset, test_labels),
          callbacks=my_callbacks,)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
